In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import datetime as dt

In [2]:
df = pd.read_csv("../input/input_1.csv")
df

,date,sentiment,clasificacion,id
0,2021-08-01,NEUTRAL,Otro,0
1,2021-08-01,NEUTRAL,Otro,1
2,2021-08-01,NEUTRAL,Otro,2
3,2021-08-01,NEUTRAL,Otro,3
4,2021-08-01,NEUTRAL,APP,4
...,...,...,...,...
464970,2022-08-10,NEUTRAL,Otro,464970
464971,2022-08-10,POSITIVE,Otro,464971
464972,2022-08-10,NEUTRAL,Otro,464972
464973,2022-08-10,NEUTRAL,Otro,464973


In [3]:
#Se crean columnas para cada sentimiento
df['MIXED'] = np.where(df['sentiment']=='MIXED', 1, 0)
df['NEGATIVE'] = np.where(df['sentiment']=='NEGATIVE', 1, 0)
df['NEUTRAL'] = np.where(df['sentiment']=='NEUTRAL', 1, 0)
df['POSITIVE'] = np.where(df['sentiment']=='POSITIVE', 1, 0)

In [5]:
#Se agrupan por fecha
df_out = df.groupby(['date']).agg(
     MIXED = ('MIXED','sum'),
     NEGATIVE = ('NEGATIVE','sum'),
     NEUTRAL = ('NEUTRAL','sum'),
     POSITIVE = ('POSITIVE','sum'),
     ).reset_index()
#Se agrega una columna del total
column_names = ['MIXED', 'NEGATIVE', 'NEUTRAL', 'POSITIVE']
df_out['ALL']= df_out[column_names].sum(axis=1)
df_out

,date,MIXED,NEGATIVE,NEUTRAL,POSITIVE,ALL
0,2021-08-01,81,231,2107,985,3404
1,2021-08-02,140,485,2033,970,3628
2,2021-08-03,198,686,1884,617,3385
3,2021-08-04,95,493,452,164,1204
4,2021-08-05,108,512,413,377,1410
...,...,...,...,...,...,...
370,2022-08-06,131,385,345,472,1333
371,2022-08-07,40,142,152,85,419
372,2022-08-08,121,381,239,42,783
373,2022-08-09,86,480,272,80,918
